<a href="https://colab.research.google.com/github/aa-pleshkov/Table_info_generation/blob/main/%D0%93%D0%B5%D0%BD%D0%B5%D1%80%D0%B0%D1%86%D0%B8%D1%8F_%D0%BE%D0%BF%D0%B8%D1%81%D0%B0%D0%BD%D0%B8%D1%8F_%D1%82%D0%B0%D0%B1%D0%BB%D0%B8%D1%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu1

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import pandas as pd
import os
import json

torch.manual_seed(42)

def model_load(model_name: str):
    """
    Загрузка LLM-модели и токенизатора с 4-битным квантованием для эффективного использования памяти.

    Args:
        model_name (str): Имя модели в Hugging Face Hub (например, 't-tech/T-lite-it-1.0').

    Returns:
        tuple: (model, tokenizer) — загруженная модель и токенизатор.

    Raises:
        ValueError: Если имя модели не указано.
        RuntimeError: Если возникла ошибка при загрузке модели.
    """
    if not model_name or not model_name.strip():
        raise ValueError("Имя модели не может быть пустым или None.")
    model_name = model_name.strip()

    try:
        # 4-битное квантование
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16,
            bnb_4bit_use_double_quant=True,
        )

        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            quantization_config=bnb_config,
            device_map="auto",
            torch_dtype=torch.bfloat16,
            trust_remote_code=False,
        )

        return model, tokenizer

    except Exception as e:
        raise RuntimeError(f"Не удалось загрузить модель '{model_name}': {e}")


def preprocess_table_info(file_path: str) -> dict:
    """
    Извлекает метаинформацию о таблице из CSV-файла.

    Args:
        file_path (str): Полный путь к CSV-файлу (например, 'data/TEST.BANK_INFORMATION.csv').

    Returns:
        Dict: Словарь с именем таблицы, списком атрибутов и примерами данных.

    Raises:
        FileNotFoundError: Если файл не найден.
        pd.errors.EmptyDataError: Если файл пуст.
        ValueError: Если имя таблицы не может быть извлечено.
    """
    print(f"Обработка файла: {file_path}")

    try:
        data = pd.read_csv(file_path)
    except FileNotFoundError:
        raise FileNotFoundError(f"Файл не найден: {file_path}")
    except pd.errors.EmptyDataError:
        raise ValueError(f"Файл пуст: {file_path}")
    except Exception as e:
        raise ValueError(f"Ошибка чтения CSV: {str(e)}")

    if data.empty:
        raise ValueError("CSV-файл содержит 0 строк данных.")

    # Извлечение имени таблицы из имени файла: prefix.table_name.csv
    filename_parts = file_path.rsplit('.', 2)
    if len(filename_parts) < 3:
        raise ValueError(f"Некорректное имя файла: {file_path}. Ожидается формат: prefix.table_name.csv")

    table_name = filename_parts[1]

    return {
        "table_name": table_name,
        "attribute_names": data.columns.tolist(),
        "example_rows": data.sample(2).to_dict(orient="list"),
    }


def make_prompt(table_info: dict) -> str:
    """
    Формирует промпт для LLM с чёткой инструкцией по генерации JSON-описания атрибутов таблицы.

    Args:
        table_info (dict): Словарь с ключами 'table_name', 'attribute_names', 'example_rows'.

    Returns:
        str: Промпт в виде строки.

    Raises:
        ValueError: Если в table_info отсутствуют необходимые поля или данные имеют неверный формат.
    """
    required_keys = ['table_name', 'attribute_names', 'example_rows']
    for key in required_keys:
        if key not in table_info:
            raise ValueError(f"Отсутствует обязательное поле в table_info: '{key}'")

    table_name = table_info['table_name']
    attribute_names = table_info['attribute_names']
    examples = table_info['example_rows']

    if not isinstance(attribute_names, (list, tuple)) or not all(isinstance(attr, str) for attr in attribute_names):
        raise ValueError("attribute_names должен быть списком строк.")
    if not isinstance(table_name, str) or not table_name.strip():
        raise ValueError("table_name должен быть непустой строкой.")
    if not isinstance(examples, dict) or not examples:
        raise ValueError("example_rows должен быть непустым словарем.")

    prompt_parts = [
        "Таблица содержит данные Россельхозбанка. Он входит в топ-5 финансово-кредитных организаций России ",
        "и предоставляет все виды банковских услуг для граждан. ",
        "Документы, на основе которых построена таблица, поступают из сферы управления счетами и денежным оборотом в банке.\n",
        f"- Имя таблицы: {table_name}",
        f"- Названия атрибутов таблицы: {attribute_names}",
        f"- Примеры строк таблицы (первые 2 строки): {examples}",
        "\nОпиши каждый атрибут таблицы. Верни ответ строго в формате JSON, где:",
        "- ключи — названия атрибутов (без изменений),",
        "- значения — краткие описания на русском языке (2–5 слов).",
        "Не добавляй пояснений, комментариев или текста вне JSON.\n",
        "Формат вывода:",
        "{",
        '  "attribute_name_1": "описание атрибута",',
        '  "attribute_name_2": "описание атрибута"',
        "}"
    ]

    prompt = "\n".join(prompt_parts)
    return prompt


def generate_response(prompt: str, model, tokenizer) -> str:
    """
    Генерирует ответ от LLM на основе промпта с использованием чат-шаблона.

    Args:
        prompt (str): Подготовленный промпт.
        model: Загруженная LLM (например, AutoModelForCausalLM).
        tokenizer: Соответствующий токенизатор (AutoTokenizer).

    Returns:
        str: Сгенерированный текст (ожидается JSON).

    Raises:
        ValueError: Если prompt пуст или не строка.
        RuntimeError: Если генерация завершилась с ошибкой.
    """
    if not isinstance(prompt, str) or not prompt.strip():
        raise ValueError("Параметр 'prompt' должен быть непустой строкой.")

    try:
        messages = [
            {"role": "system", "content": "Твоя задача — составлять точные и понятные описания атрибутов табличных данных."},
            {"role": "user", "content": prompt.strip()}
        ]

        # Чат-шаблон
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )

        # Токенизация
        model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=512,
            do_sample=False,
            num_beams=1,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            temperature=0.01,
            top_p=0.95,
        )

        # Ответ модели
        input_length = model_inputs.input_ids.shape[1]
        generated_ids = [output_ids[input_length:] for output_ids in generated_ids]

        # Декодирование
        response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

        return response.strip()

    except Exception as e:
        raise RuntimeError(f"Ошибка при генерации ответа: {e}")


def process_file(file_path: str, model, tokenizer) -> tuple:
    """
    Обрабатывает один файл с уже загруженной моделью.

    Args:
        file_path (str): Путь к CSV-файлу.
        model: Загруженная LLM.
        tokenizer: Соответствующий токенизатор.

    Returns:
        tuple: (result: str, table_info: dict) — сгенерированный ответ и метаданные таблицы.

    В случае ошибки:
        - Если ошибка на этапе preprocess — выбрасывается исключение.
        - Если ошибка в make_prompt или generate_response — возвращается строка с описанием ошибки.
        (Сохраняем логику возврата кортежа, как в оригинале.)
    """
    try:
        table_info = preprocess_table_info(file_path)
    except Exception as e:
        raise RuntimeError(f"Ошибка при предобработке файла '{file_path}': {e}")

    try:
        prompt = make_prompt(table_info)
    except Exception as e:
        error_msg = f"Ошибка при создании промпта: {e}"
        print(f"{error_msg}")
        return error_msg, table_info

    try:
        result = generate_response(prompt, model, tokenizer)
    except Exception as e:
        error_msg = f"Ошибка при генерации ответа модели: {e}"
        print(f"{error_msg}")
        return error_msg, table_info

    return result, table_info


def main(folder_path: str, model, tokenizer) -> dict:
    """
    Обрабатывает все CSV-файлы в указанной папке: извлекает информацию,
    генерирует описания атрибутов с помощью LLM и возвращает словарь с результатами.

    Args:
        folder_path (str): Путь к папке с CSV-файлами.
        model: Загруженная LLM.
        tokenizer: Соответствующий токенизатор.

    Returns:
        dict: Словарь, где ключ — имя таблицы, значение — распарсенный JSON с описаниями атрибутов.
              В случае ошибки файл пропускается, процесс продолжается.
    """
    res = {}
    try:
        file_names = os.listdir(folder_path)
    except Exception as e:
        print(f"Ошибка при чтении папки '{folder_path}': {e}")
        return res

    for f in file_names:
        if f.endswith('.csv'):
            file_path = os.path.join(folder_path, f)
            try:
                result, table_info = process_file(file_path, model, tokenizer)
                table_name = table_info['table_name']

                # Очистка результата от маркеров кода
                cleaned_result = result.strip()
                if cleaned_result.startswith('```json'):
                    cleaned_result = cleaned_result[7:].strip()
                elif cleaned_result.startswith('```'):
                    cleaned_result = cleaned_result[3:].strip()
                if cleaned_result.endswith('```'):
                    cleaned_result = cleaned_result[:-3].strip()

                try:
                    parsed_result = json.loads(cleaned_result)
                    res[table_name] = parsed_result
                except json.JSONDecodeError as e:
                    print(f"Ошибка парсинга JSON в файле '{f}': {e}")
                    continue

            except Exception as e:
                print(f"Пропущен файл '{f}': ошибка при обработке — {e}")
                continue

    return res

In [ ]:
'''
# Запуск обработки
if __name__ == "__main__":

    FOLDER_PATH = "data"          # Путь к папке с CSV-файлами
    MODEL_NAME = "t-tech/T-lite-it-1.0"  # Имя модели
    MAX_FILES = None                # Обработать все файлы (или указать необходимое число для теста)

    if not os.path.exists(FOLDER_PATH):
        raise FileNotFoundError(f"Папка не найдена: {FOLDER_PATH}")

    try:
        model, tokenizer = model_load(MODEL_NAME)
        results = main(FOLDER_PATH, model, tokenizer)

        # Сохранение результата
        output_file = "attribute_descriptions.json"
        with open(output_file, "w", encoding="utf-8") as f:
            json.dump(results, f, ensure_ascii=False, indent=2)

    except Exception as e:
        print(f"Ошибка при выполнении: {e}")
        raise
'''

In [ ]:
model, tokenizer = model_load('t-tech/T-lite-it-1.0')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.33G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [ ]:
res = main('data', model, tokenizer)

output_file = "attribute_descriptions.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(res, f, ensure_ascii=False, indent=2)

# print(res)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Обработка файла: data/TEST.BANK_INFORMATION.csv


In [ ]:
import transformers
import torch
import pandas as pd

print("transformers:", transformers.__version__)
print("torch:", torch.__version__)
print("pandas:", pd.__version__)

transformers: 4.55.0
torch: 2.6.0+cu124
pandas: 2.2.2


/tmp/ipython-input-31738736.py:11: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


bitsandbytes: версию нужно уточнить


- название источника(бд, хост, логин, название таблицы) - вход. отдельная функция